## Importing the Libraries

In [ ]:
import numpy as np
from tensorflow.python.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from tensorflow.python.keras.models import Sequential
import matplotlib.pyplot as plt
%matplotlib inline

## Importing variables from the preprocessing notebook as csv files


In [ ]:
X_train = np.loadtxt('X_train.csv', delimiter=',')
X_test = np.loadtxt('X_test.csv', delimiter=',')
y_train = np.loadtxt('y_train.csv', delimiter=',')
y_test = np.loadtxt('y_test.csv', delimiter=',')

In [ ]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)

## CNN Model

In [ ]:
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Build a simple 1D CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification


In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model and store the history
history_cnn = model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_split=0.2)